In [2]:
import torch
import torchvision
from tqdm import tqdm
from typing import List, Dict
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation,PacketFlowRepressentation,TimeslotRepresentation
from flowprintOptimal.sekigo.dataAnalysis.vNATDataFrameProcessor import VNATDataFrameProcessor
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
import random
from flowprintOptimal.sekigo.flowUtils.flowDatasets import PacketFlowDataset, BaseFlowDataset, DDQNActivityDataset
from torch.utils.data import Dataset,DataLoader
from torchsampler import ImbalancedDatasetSampler
from sklearn.model_selection import train_test_split
from flowprintOptimal.sekigo.flowUtils.commons import normalizePacketRep
import os
from joblib import Parallel, delayed
from flowprintOptimal.sekigo.flowUtils.commons import saveFlows,loadFlows
from flowprintOptimal.sekigo.dataAnalysis.dataFrameProcessor import UTMobileNetProcessor
from flowprintOptimal.sekigo.flowUtils.dataGetterV2 import readTrainTestOODFlows
from sklearn.metrics import confusion_matrix
import json
from flowprintOptimal.sekigo.modeling.trainers import NNClassificationTrainer
from flowprintOptimal.sekigo.modeling.neuralNetworks import LSTMNetwork,TransformerGenerator,CNNNetwork1D, LSTMDuelingNetwork
from flowprintOptimal.sekigo.modeling.loggers import Logger
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
from flowprintOptimal.sekigo.earlyClassification.DQL.core import MemoryElement,Rewarder,State
from flowprintOptimal.sekigo.earlyClassification.DQL.memoryFiller import MemoryFiller
from flowprintOptimal.sekigo.earlyClassification.DQL.datasets import MemoryDataset
from flowprintOptimal.sekigo.earlyClassification.DQL.trainers import EarlyClassificationtrainer
from flowprintOptimal.sekigo.utils.documentor import Documenter
from flowprintOptimal.sekigo.utils.evaluations import Evaluator,EarlyEvaluation
import warnings
from copy import deepcopy
import torch.nn as nn



warnings.filterwarnings('ignore')

In [4]:
network = LSTMDuelingNetwork(lstm_hidden_size= 256,lstm_input_size= 10, layers= 2,output_dim= 7)

In [5]:
example = torch.rand(1,20,10)

In [7]:
traced_script_module = torch.jit.trace(network, example)

In [8]:
traced_script_module(example)

(tensor([[-0.1580, -0.1386, -0.0693, -0.0597, -0.1530, -0.0366, -0.0358]],
        grad_fn=<AddBackward0>),
 tensor([[ 3.2210e-03,  6.2879e-02, -7.3430e-02,  3.1444e-02, -3.6714e-03,
          -1.6759e-03, -5.1150e-03,  1.4612e-02, -5.3363e-02,  3.7192e-02,
           2.4859e-02, -1.8466e-02,  5.1733e-02,  6.6292e-02,  3.5652e-02,
           3.5038e-02,  4.2754e-02, -6.7033e-03, -3.3303e-02, -1.6958e-02,
           5.1682e-03,  4.7919e-02,  1.5484e-03,  2.2460e-02, -4.1624e-02,
          -2.1634e-02, -6.5866e-03,  4.9064e-03,  4.2124e-02, -6.1773e-02,
          -1.7218e-02, -2.7405e-02, -1.7912e-02,  4.8908e-02,  1.1041e-02,
          -2.1202e-03, -2.4233e-02, -2.9144e-02,  7.0681e-03, -2.7705e-02,
           7.2039e-04, -1.6173e-02, -1.5015e-02,  5.5594e-02, -9.7403e-03,
           2.5633e-02, -7.4461e-03,  4.1674e-02,  3.1076e-02,  2.1510e-02,
          -2.4849e-02,  1.0311e-02,  4.9186e-04,  4.0195e-03, -5.2153e-02,
           7.2342e-02, -4.5654e-02,  5.2210e-03, -7.3309e-03, -8.70

In [9]:
traced_script_module.save("models/traced_model.pt")